In [ ]:
import polars as pl
from math import sqrt
import queue
import random
import rtsvg
rt = rtsvg.RACETrack()
import os
import kagglehub
path = kagglehub.dataset_download("konivat/tree-of-life")
dfs = []
for _filename_ in os.listdir(path):
    df = pl.read_csv(os.path.join(path, _filename_))
    dfs.append(df)

xs, ys, ws, hs = -20.0, -20.0, 40.0, 40.0
def svgCircles(_pkd_, _chn_, _color_='#000000', extra_lines=None, w=250, h=250):
    svg = [f'<svg x="0" y="0" width="{w}" height="{h}" viewBox="{xs} {ys} {ws} {hs}" xmlns="http://www.w3.org/2000/svg">']
    svg.append(f'<rect x="{xs}" y="{ys}" width="{ws}" height="{hs}" fill="#ffffff" />')
    svg.append(f'<line x1="-1" y1="-1" x2="1" y2="1" stroke="black" stroke-width="0.2" />')
    svg.append(f'<line x1="1" y1="-1" x2="-1" y2="1" stroke="black" stroke-width="0.2" />')
    for i in range(len(_pkd_)):
        _c_ = _pkd_[i]
        svg.append(f'<circle cx="{_c_[0]}" cy="{_c_[1]}" r="{_c_[2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.2" />')
    _index_ = list(_chn_.keys())[0]
    for i in range(len(_chn_.keys())+1):
        _index_next_ = _chn_[_index_]
        xy0, xy1 = _pkd_[_index_], _pkd_[_index_next_]
        svg.append(f'<line x1="{xy0[0]}" y1="{xy0[1]}" x2="{xy1[0]}" y2="{xy1[1]}" stroke="{_color_}" stroke-width="0.2" />')
        uv       = rt.unitVector((xy0, xy1))
        perp     = (-uv[1], uv[0])
        svg.append(f'<line x1="{xy1[0]}" y1="{xy1[1]}" x2="{xy1[0] - 1*uv[0] + 0.5*perp[0]}" y2="{xy1[1] - 1*uv[1] + 0.5*perp[1]}" stroke="{_color_}" stroke-width="0.1" />')
        svg.append(f'<line x1="{xy1[0]}" y1="{xy1[1]}" x2="{xy1[0] - 1*uv[0] - 0.5*perp[0]}" y2="{xy1[1] - 1*uv[1] - 0.5*perp[1]}" stroke="{_color_}" stroke-width="0.1" />')
        _index_  = _index_next_
    if extra_lines is not None: svg.extend(extra_lines)
    svg.append('</svg>')
    return ''.join(svg)

In [ ]:
#
# Implementation of the following paper:
#
#@inproceedings{10.1145/1124772.1124851,
#               author = {Wang, Weixin and Wang, Hui and Dai, Guozhong and Wang, Hongan},
#               title = {Visualization of large hierarchical data by circle packing},
#               year = {2006},
#               isbn = {1595933727},
#               publisher = {Association for Computing Machinery},
#               address = {New York, NY, USA},
#               url = {https://doi.org/10.1145/1124772.1124851},
#               doi = {10.1145/1124772.1124851},
#               abstract = {In this paper a novel approach is described for tree visualization using nested circles. 
#                           The brother nodes at the same level are represented by externally tangent circles; 
#                           the tree nodes at different levels are displayed by using 2D nested circles or 3D nested cylinders. 
#                           A new layout algorithm for tree structure is described. It provides a good overview for large data sets. 
#                           It is easy to see all the branches and leaves of the tree. The new method has been applied to the 
#                           visualization of file systems.},
#               booktitle = {Proceedings of the SIGCHI Conference on Human Factors in Computing Systems},
#               pages = {517–520},
#               numpages = {4},
#               keywords = {tree visualization, nested circles, file system, circle packing},
#               location = {Montr\'{e}al, Qu\'{e}bec, Canada},
#               series = {CHI '06} }
#
def packCircles(self, circles, epsilon=0.01):
    if   len(circles) == 0: return [], {}, {}, []
    elif len(circles) == 1: return [(0.0, 0.0, circles[0][2])], {}, {}, []
    elif len(circles) == 2: return [(0.0, 0.0, circles[0][2]), (circles[0][2] + circles[1][2], 0.0, circles[1][2])], {}, {}, []
    else:
        progression = []
        placed      = []
        placed.append((0.0, 0.0, circles[0][2]))
        placed.append((circles[0][2] + circles[1][2], 0.0, circles[1][2]))
        c0, c1, r = placed[0], placed[1], circles[2][2]
        xy0, xy1 = self.overlappingCirclesIntersections((c0[0],c0[1],c0[2]+r),(c1[0],c1[1],c1[2]+r))
        placed.append((xy0[0], xy0[1], r))
        # Create the forward and backward chains
        fwd,    bck    = {}, {}
        fwd[0], bck[0] = 1, 2
        fwd[1], bck[1] = 2, 0
        fwd[2], bck[2] = 0, 1

        def validateChains(ll=None, name=None):
            if ll is None:
                validateChains(ll=fwd, name='fwd')
                validateChains(ll=bck, name='bck')
            else:
                print(name, ll.keys())
                i = j = list(ll.keys())[0]
                print(f'{i} -> ', end='')
                j = ll[i]
                while j != i:
                    print(f'{j} -> ', end='')
                    j = ll[j]
                print(i)

        # List all of the circles in the chain
        in_chain = set([0, 1, 2])
        # Create a priority queue -- this will return the closest point to the origin
        nearest  = queue.PriorityQueue()
        nearest.put((0.0, 0))
        nearest.put((sqrt(placed[1][0]**2 + placed[1][1]**2), 1))
        nearest.put((sqrt(placed[2][0]**2 + placed[2][1]**2), 2))
        progression.append(svgCircles(placed, fwd))
        for i in range(3, len(circles)):
            c = circles[i]
            while nearest.queue[0][1] not in in_chain: nearest.get() # find the next nearest circle that is still in the chain
            cm_i, cn_i = nearest.queue[0][1], fwd[nearest.queue[0][1]]
            cm,   cn   = placed[cm_i], placed[cn_i]

            circle_placed = False
            while circle_placed == False:
                xy0,  xy1  = self.overlappingCirclesIntersections((cm[0],cm[1],cm[2]+c[2]),(cn[0],cn[1],cn[2]+c[2]))
                progression.append(svgCircles(placed, fwd, extra_lines=[f'<circle cx="{xy0[0]}" cy="{xy0[1]}" r="0.3" fill="#ff0000" stroke="#ff0000" stroke-width=".1" />',
                                                                        f'<circle cx="{xy1[0]}" cy="{xy1[1]}" r="0.3" fill="#00ff00" stroke="#00ff00" stroke-width=".1" />']))
                c_possible          = (xy1[0], xy1[1], c[2])
                cj                  = None
                cj_i                = None
                cj_i_after_cn_count = None
                nxt_i               = fwd[cn_i]
                after_cn            = 1
                while nxt_i != cm_i:
                    if self.circlesOverlap((c_possible[0], c_possible[1], c_possible[2]-epsilon), placed[nxt_i]):
                        progression.append(svgCircles(placed, fwd, extra_lines=[f'<circle cx="{c_possible[0]}" cy="{c_possible[1]}" r="{c_possible[2]}" fill="none" stroke="#ff0000" stroke-width="0.5" />']))
                        cj, cj_i, cj_i_after_cn_count = placed[nxt_i], nxt_i, after_cn
                    nxt_i     = fwd[nxt_i]
                    after_cn += 1
                if cj is None:
                    placed.append((xy1[0], xy1[1], c[2]))
                    fwd[i]    = cn_i
                    fwd[cm_i] = i
                    bck[i]    = cm_i
                    bck[cn_i] = i
                    in_chain.add(i)
                    nearest.put((sqrt(c_possible[0]**2 + c_possible[1]**2), i))
                    circle_placed = True
                    validateChains()
                elif cj_i_after_cn_count < len(in_chain)/2: # After cn
                    print('after')
                    j = cn_i
                    while j != cj_i:
                        in_chain.remove(j)
                        j_last = j
                        j      = fwd[j]
                        del fwd[j_last]
                        del bck[j_last]
                    cn, cn_i = cj, cj_i
                    fwd[cm_i] = cn_i
                    bck[cn_i] = cm_i
                    validateChains()
                else:                                       # Before 
                    print('before')
                    j = cj_i
                    while j != cn_i:
                        in_chain.remove(j)
                        j_last = j
                        j      = fwd[j]
                        del fwd[j_last]
                        del bck[j_last]
                    cm, cm_i = cj, cj_i
                    fwd[cm_i] = cn_i
                    bck[cn_i] = cm_i
                    validateChains()

        progression.append(svgCircles(placed, fwd))
        return placed, fwd, bck, progression

#_circles_ = []
#for i in range(11): _circles_.append((0.0, 0.0, 1.0 + 3*random.random()))
#_packed_, _fwd_, _bck_, _tiles_ = packCircles(rt, _circles_)
#rt.table(_tiles_, per_row=6, spacer=10)

In [ ]:
#rt.tile([svgCircles(_packed_, _fwd_, w=500, h=500), 
#         svgCircles(_packed_, _fwd_, '#ff0000', w=500, h=500)], spacer=10)

In [ ]:
#
# Implementation of the following paper:
#
#@inproceedings{10.1145/1124772.1124851,
#               author = {Wang, Weixin and Wang, Hui and Dai, Guozhong and Wang, Hongan},
#               title = {Visualization of large hierarchical data by circle packing},
#               year = {2006},
#               isbn = {1595933727},
#               publisher = {Association for Computing Machinery},
#               address = {New York, NY, USA},
#               url = {https://doi.org/10.1145/1124772.1124851},
#               doi = {10.1145/1124772.1124851},
#               abstract = {In this paper a novel approach is described for tree visualization using nested circles. 
#                           The brother nodes at the same level are represented by externally tangent circles; 
#                           the tree nodes at different levels are displayed by using 2D nested circles or 3D nested cylinders. 
#                           A new layout algorithm for tree structure is described. It provides a good overview for large data sets. 
#                           It is easy to see all the branches and leaves of the tree. The new method has been applied to the 
#                           visualization of file systems.},
#               booktitle = {Proceedings of the SIGCHI Conference on Human Factors in Computing Systems},
#               pages = {517–520},
#               numpages = {4},
#               keywords = {tree visualization, nested circles, file system, circle packing},
#               location = {Montr\'{e}al, Qu\'{e}bec, Canada},
#               series = {CHI '06} }
#
def packCircles(self, circles, epsilon=0.01):
    if   len(circles) == 0: return [], {}, {}, []
    elif len(circles) == 1: return [(0.0, 0.0, circles[0][2])], {}, {}, []
    elif len(circles) == 2: return [(0.0, 0.0, circles[0][2]), (circles[0][2] + circles[1][2], 0.0, circles[1][2])], {}, {}, []
    else:
        progression = []
        placed      = []
        placed.append((0.0, 0.0, circles[0][2]))
        placed.append((circles[0][2] + circles[1][2], 0.0, circles[1][2]))
        c0, c1, r = placed[0], placed[1], circles[2][2]
        xy0, xy1 = self.overlappingCirclesIntersections((c0[0],c0[1],c0[2]+r),(c1[0],c1[1],c1[2]+r))
        placed.append((xy0[0], xy0[1], r))
        # Create the forward and backward chains
        fwd    = {}
        fwd[0] = 1
        fwd[1] = 2
        fwd[2] = 0

        def validateChains(ll=fwd, name='fwd'):
            print(name, ll.keys())
            i = j = list(ll.keys())[0]
            print(f'{i} -> ', end='')
            j = ll[i]
            while j != i:
                print(f'{j} -> ', end='')
                j = ll[j]
            print(i)

        # Create a priority queue -- this will return the closest point to the origin
        nearest  = queue.PriorityQueue()
        nearest.put((0.0, 0))
        nearest.put((sqrt(placed[1][0]**2 + placed[1][1]**2), 1))
        nearest.put((sqrt(placed[2][0]**2 + placed[2][1]**2), 2))
        progression.append(svgCircles(placed, fwd))
        for i in range(3, len(circles)):
            c = circles[i]
            while nearest.queue[0][1] not in fwd.keys(): nearest.get() # find the next nearest circle that is still in the chain
            cm_i, cn_i = nearest.queue[0][1], fwd[nearest.queue[0][1]]
            cm,   cn   = placed[cm_i], placed[cn_i]

            circle_placed = False
            while circle_placed == False:
                xy0,  xy1  = self.overlappingCirclesIntersections((cm[0],cm[1],cm[2]+c[2]),(cn[0],cn[1],cn[2]+c[2]))
                progression.append(svgCircles(placed, fwd, extra_lines=[f'<circle cx="{xy0[0]}" cy="{xy0[1]}" r="0.3" fill="#ff0000" stroke="#ff0000" stroke-width=".1" />',
                                                                        f'<circle cx="{xy1[0]}" cy="{xy1[1]}" r="0.3" fill="#00ff00" stroke="#00ff00" stroke-width=".1" />']))
                c_possible          = (xy1[0], xy1[1], c[2])
                cj                  = None
                cj_i                = None
                cj_i_after_cn_count = None
                nxt_i               = fwd[cn_i]
                after_cn            = 1
                while nxt_i != cm_i:
                    if self.circlesOverlap((c_possible[0], c_possible[1], c_possible[2]-epsilon), placed[nxt_i]):
                        progression.append(svgCircles(placed, fwd, extra_lines=[f'<circle cx="{c_possible[0]}" cy="{c_possible[1]}" r="{c_possible[2]}" fill="none" stroke="#ff0000" stroke-width="0.5" />']))
                        cj, cj_i, cj_i_after_cn_count = placed[nxt_i], nxt_i, after_cn
                    nxt_i     = fwd[nxt_i]
                    after_cn += 1
                if cj is None:
                    placed.append((xy1[0], xy1[1], c[2]))
                    fwd[i]    = cn_i
                    fwd[cm_i] = i
                    nearest.put((sqrt(c_possible[0]**2 + c_possible[1]**2), i))
                    circle_placed = True
                elif cj_i_after_cn_count < len(fwd.keys())/2: # After cn
                    j = cn_i
                    while j != cj_i:
                        j_last = j
                        j      = fwd[j]
                        del fwd[j_last]
                    cn, cn_i = cj, cj_i
                    fwd[cm_i] = cn_i
                else:                                       # Before 
                    j = cj_i
                    while j != cn_i:
                        j_last = j
                        j      = fwd[j]
                        del fwd[j_last]
                    cm, cm_i = cj, cj_i
                    fwd[cm_i] = cn_i

        progression.append(svgCircles(placed, fwd))
        return placed, fwd, progression

#_circles_ = []
#for i in range(20): _circles_.append((0.0, 0.0, 0.2 + 0.3*random.random()))
#_packed_, _fwd_, _tiles_ = packCircles(rt, _circles_)
#rt.table(_tiles_, per_row=6, spacer=10)

In [ ]:
#
# Implementation of the following paper:
#
#@inproceedings{10.1145/1124772.1124851,
#               author = {Wang, Weixin and Wang, Hui and Dai, Guozhong and Wang, Hongan},
#               title = {Visualization of large hierarchical data by circle packing},
#               year = {2006},
#               isbn = {1595933727},
#               publisher = {Association for Computing Machinery},
#               address = {New York, NY, USA},
#               url = {https://doi.org/10.1145/1124772.1124851},
#               doi = {10.1145/1124772.1124851},
#               abstract = {In this paper a novel approach is described for tree visualization using nested circles. 
#                           The brother nodes at the same level are represented by externally tangent circles; 
#                           the tree nodes at different levels are displayed by using 2D nested circles or 3D nested cylinders. 
#                           A new layout algorithm for tree structure is described. It provides a good overview for large data sets. 
#                           It is easy to see all the branches and leaves of the tree. The new method has been applied to the 
#                           visualization of file systems.},
#               booktitle = {Proceedings of the SIGCHI Conference on Human Factors in Computing Systems},
#               pages = {517–520},
#               numpages = {4},
#               keywords = {tree visualization, nested circles, file system, circle packing},
#               location = {Montr\'{e}al, Qu\'{e}bec, Canada},
#               series = {CHI '06} }
#
def packCircles(self, circles, epsilon=0.01):
    if   len(circles) == 0: return []
    elif len(circles) == 1: return [(0.0, 0.0, circles[0][2])]
    elif len(circles) == 2: return [(0.0, 0.0, circles[0][2]), (circles[0][2] + circles[1][2], 0.0, circles[1][2])]
    else:
        placed = []
        placed.append((0.0, 0.0, circles[0][2]))
        placed.append((circles[0][2] + circles[1][2], 0.0, circles[1][2]))
        c0, c1, r = placed[0], placed[1], circles[2][2]
        xy0, xy1 = self.overlappingCirclesIntersections((c0[0],c0[1],c0[2]+r),(c1[0],c1[1],c1[2]+r))
        placed.append((xy0[0], xy0[1], r))
        # Create the forward and backward chains
        fwd    = {}
        fwd[0] = 1
        fwd[1] = 2
        fwd[2] = 0

        def validateChains(ll=fwd, name='fwd'):
            print(name, ll.keys())
            i = j = list(ll.keys())[0]
            print(f'{i} -> ', end='')
            j = ll[i]
            while j != i:
                print(f'{j} -> ', end='')
                j = ll[j]
            print(i)

        # Create a priority queue -- this will return the closest point to the origin
        nearest  = queue.PriorityQueue()
        nearest.put((0.0, 0))
        nearest.put((sqrt(placed[1][0]**2 + placed[1][1]**2), 1))
        nearest.put((sqrt(placed[2][0]**2 + placed[2][1]**2), 2))
        for i in range(3, len(circles)):
            c = circles[i]
            while nearest.queue[0][1] not in fwd.keys(): nearest.get() # find the next nearest circle that is still in the chain
            cm_i, cn_i = nearest.queue[0][1], fwd[nearest.queue[0][1]]
            cm,   cn   = placed[cm_i], placed[cn_i]

            circle_placed = False
            while circle_placed == False:
                xy0,  xy1  = self.overlappingCirclesIntersections((cm[0],cm[1],cm[2]+c[2]),(cn[0],cn[1],cn[2]+c[2]))
                c_possible          = (xy1[0], xy1[1], c[2])
                cj                  = None
                cj_i                = None
                cj_i_after_cn_count = None
                nxt_i               = fwd[cn_i]
                after_cn            = 1
                while nxt_i != cm_i:
                    if self.circlesOverlap((c_possible[0], c_possible[1], c_possible[2]-epsilon), placed[nxt_i]):
                        cj, cj_i, cj_i_after_cn_count = placed[nxt_i], nxt_i, after_cn
                    nxt_i     = fwd[nxt_i]
                    after_cn += 1
                if cj is None:
                    placed.append((xy1[0], xy1[1], c[2]))
                    fwd[i]    = cn_i
                    fwd[cm_i] = i
                    nearest.put((sqrt(c_possible[0]**2 + c_possible[1]**2), i))
                    circle_placed = True
                elif cj_i_after_cn_count < len(fwd.keys())/2: # After cn
                    j = cn_i
                    while j != cj_i:
                        j_last = j
                        j      = fwd[j]
                        del fwd[j_last]
                    cn, cn_i = cj, cj_i
                    fwd[cm_i] = cn_i
                else:                                       # Before 
                    j = cj_i
                    while j != cn_i:
                        j_last = j
                        j      = fwd[j]
                        del fwd[j_last]
                    cm, cm_i = cj, cj_i
                    fwd[cm_i] = cn_i

        return placed

_circles_ = [(0.0, 0.0, 0.3163975984875254),
             (3.0216854230135723, 0.0, 2.7052878245260468),
             (-2.071830337728682, 2.61624257763915, 3.020847698667733),
             (4.815313677427861, -0.4559745206391095, 4.520456603935356),
             (2.167306469457607, 0.1973261971893452, 1.8598732855363374),
             (-0.1971444335599162, -1.1728229236754637, 0.872879298615215),
             (-3.5789985477050266, -3.388515280368342, 3.1701678033695564),
             (2.6423374410704636, -4.864059456287823, 3.223755256874679),
             (4.851801893718894, 6.332921430507575, 4.8372936192075775),
             (4.550309177613312, 0.8838991241432657, 0.6200630626596191)]
# for i in range(10): _circles_.append((0.0, 0.0, 0.2 + 5.0*random.random()))
_packed_ = packCircles(rt, _circles_[:4])
def renderCircles(cs):
    x0, y0, x1, y1 = cs[0][0] - cs[0][2], cs[0][1] - cs[0][2], cs[0][0] + cs[0][2], cs[0][1] + cs[0][2]
    for i in range(1, len(cs)):
        x0, y0, x1, y1 = min(x0, cs[i][0] - cs[i][2]), min(y0, cs[i][1] - cs[i][2]), max(x1, cs[i][0] + cs[i][2]), max(y1, cs[i][1] + cs[i][2])
    _svg_ = [f'<svg x="0" y="0" width="400" height="400" viewBox="{x0} {y0} {x1 - x0} {y1 - y0}" xmlns="http://www.w3.org/2000/svg">']
    _svg_.append(f'<rect x="{x0}" y="{y0}" width="{x1 - x0}" height="{y1 - y0}" fill="#ffffff" />')
    for i in range(len(cs)):
        _c_ = cs[i]
        _svg_.append(f'<circle cx="{_c_[0]}" cy="{_c_[1]}" r="{_c_[2]}" fill="none" stroke="black" stroke-width="0.2" />')
    _svg_.append('</svg>')
    return ''.join(_svg_)
rt.tile([renderCircles(_packed_)])